<a href="https://colab.research.google.com/github/Aadarsh-1210/hackathon/blob/main/2nd_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyQt5 opencv-python torch
import sys
import cv2
import torch
import numpy as np
import json
import xml.etree.ElementTree as ET
import csv
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QVBoxLayout, QHBoxLayout, QWidget, QFileDialog, QListWidget
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
from PyQt5.QtCore import Qt, QRect

# Step 1: Load the image using OpenCV, displaying it in the GUI
class LabelMaster(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Label Master")
        self.setGeometry(100, 100, 1000, 800)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QHBoxLayout(self.central_widget)

        self.image_label = QLabel(self)
        self.image_label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.image_label, 3)

        self.sidebar = QWidget()
        self.sidebar_layout = QVBoxLayout(self.sidebar)
        self.layout.addWidget(self.sidebar, 1)

        self.load_button = QPushButton("Load Image", self)
        self.load_button.clicked.connect(self.load_image)
        self.sidebar_layout.addWidget(self.load_button)

        self.detect_button = QPushButton("Detect Objects", self)
        self.detect_button.clicked.connect(self.detect_objects)
        self.sidebar_layout.addWidget(self.detect_button)

        self.annotation_list = QListWidget(self)
        self.sidebar_layout.addWidget(self.annotation_list)

        self.export_button = QPushButton("Export Annotations", self)
        self.export_button.clicked.connect(self.export_annotations)
        self.sidebar_layout.addWidget(self.export_button)

        self.current_image = None
        self.annotations = []
        self.model = self.load_yolo_model()


    # ... (other methods remain the same)

    def load_image(self, file_name=None):
        if file_name is None:
            file_name, _ = QFileDialog.getOpenFileName(self, "Open Image File", "", "Images (*.png *.jpg *.bmp)")

        if file_name:
            self.current_image = cv2.imread(file_name)
            if self.current_image is not None:
                self.display_image(self.current_image)
                print(f"Loaded image: {file_name}")
            else:
                print(f"Failed to load image: {file_name}")

    def display_image(self, img):
        height, width, channel = img.shape
        bytes_per_line = 3 * width
        q_img = QImage(img.data, width, height, bytes_per_line, QImage.Format_RGB888).rgbSwapped()
        pixmap = QPixmap.fromImage(q_img)
        self.image_label.setPixmap(pixmap.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

    # Step 2: Pass the image through a pre-trained object detection model
    def load_yolo_model(self):
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        model.eval()
        return model

    def detect_objects(self):
        if self.current_image is None:
            return

        img = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2RGB)
        results = self.model(img)

        self.annotations = []
        for *xyxy, conf, cls in results.xyxy[0]:
            x1, y1, x2, y2 = map(int, xyxy)
            label = f"{results.names[int(cls)]} {conf:.2f}"
            self.annotations.append({"label": label, "bbox": [x1, y1, x2-x1, y2-y1]})

        self.draw_annotations()
        self.update_annotation_list()

    # Step 3: Display predictions in the GUI and provide options for users to edit them
    def draw_annotations(self):
        if self.current_image is None:
            return

        annotated_image = self.current_image.copy()
        for ann in self.annotations:
            x, y, w, h = ann["bbox"]
            cv2.rectangle(annotated_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(annotated_image, ann["label"], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

        self.display_image(annotated_image)

    def update_annotation_list(self):
        self.annotation_list.clear()
        for i, ann in enumerate(self.annotations):
            self.annotation_list.addItem(f"{i+1}. {ann['label']}")

    # Step 4: Implement export functionality
    def export_annotations(self):
        if not self.annotations:
            return

        file_name, _ = QFileDialog.getSaveFileName(self, "Save Annotations", "", "JSON (*.json);;XML (*.xml);;CSV (*.csv)")
        if file_name:
            if file_name.endswith('.json'):
                self.export_json(file_name)
            elif file_name.endswith('.xml'):
                self.export_xml(file_name)
            elif file_name.endswith('.csv'):
                self.export_csv(file_name)

    def export_json(self, file_name):
        with open(file_name, 'w') as f:
            json.dump(self.annotations, f)

    def export_xml(self, file_name):
        root = ET.Element("annotations")
        for ann in self.annotations:
            obj = ET.SubElement(root, "object")
            ET.SubElement(obj, "label").text = ann["label"]
            bbox = ET.SubElement(obj, "bndbox")
            x, y, w, h = ann["bbox"]
            ET.SubElement(bbox, "xmin").text = str(x)
            ET.SubElement(bbox, "ymin").text = str(y)
            ET.SubElement(bbox, "xmax").text = str(x + w)
            ET.SubElement(bbox, "ymax").text = str(y + h)
        tree = ET.ElementTree(root)
        tree.write(file_name)

    def export_csv(self, file_name):
        with open(file_name, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["label", "x", "y", "width", "height"])
            for ann in self.annotations:
                writer.writerow([ann["label"]] + ann["bbox"])


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = LabelMaster()

    # Load a specific image at startup
    window.load_image("C:/Users/Lenovo/OneDrive/Pictures")

    window.show()
    sys.exit(app.exec_())